In [1]:
from minimax import minimax
from transposition_table import ZobristBoard
from gaviota import get_move_from_table
import chess, chess.pgn
from IPython.display import display, SVG
import chess.svg
import chess.gaviota

# ====== CONFIG ======
TB_DIR = "../tablebases/gaviota"
STORAGE = "../archive"

SAVE = False

def save_function(game, storage, filename):
    with open(f"{storage}/{filename}", "w", encoding="utf-8") as pgn_file:
        print(game, file=pgn_file)

SAVE_CONFIG = {
    "event": "Piękny Endgame",
    "date": "2025.10.01",
    "filename": "endgame_example.pgn",
    "setup": True,
    "save_function": save_function
}

# ====================

# 4k3/3n1b2/8/8/7R/R7/2K5/8 w - - 0 1
board = ZobristBoard("8/2r5/3kn3/2b5/8/3K4/1R6/8 w - - 0 1")
white = True

display(SVG(chess.svg.board(board, size=400)))

# Ustawienia zapisu partii
if SAVE:
    game = chess.pgn.Game()
    game.headers["Event"] = SAVE_CONFIG["event"]
    game.headers["Date"] = SAVE_CONFIG["date"]
    game.setup = SAVE_CONFIG["setup"]
    game.headers["FEN"] = board.fen()
else:
    game = chess.pgn.Game()

node = game
move_count = 0

while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move = get_move_from_table(board, tb)
            print("Move from tablebase:", move)
        else:
            score, move = minimax(board, 6, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        node = node.add_variation(move)
        board.push(move)
        move_count += 1

        white = not white
        display(SVG(chess.svg.board(board, size=400,arrows=[chess.svg.Arrow(move.from_square, move.to_square, color="#fc681fcc")],)))

print("Game over:", board.result())

if SAVE:
    SAVE_CONFIG["save_function"](game, STORAGE, SAVE_CONFIG["filename"])

ImportError: attempted relative import with no known parent package